In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk import tokenize
# nltk.download('punkt')

## Gathering data from different papers about mountains

In [2]:
with open("papers_about_mountains.txt",'r') as file:
    text = file.read()

In [3]:
mountains = []
with open("list_of_mountains.txt",'r') as file: 
    for line in file:
        if line.strip()!="":
            mountains.append(line.strip())

In [4]:
sentences_list = tokenize.sent_tokenize(text)

In [5]:
sentenses_num = []
words_list = []
labels = []
for idx, sentence in enumerate(sentences_list):
    words = nltk.word_tokenize(sentence)
    for w in words:
        words_list.append(w)
        sentenses_num.append(idx+1)
        labels.append(1 if w in mountains else 0)

In [6]:
# dict_data = {}
# dict_data['Sentence']=sentenses_num
# dict_data['Word']=words_list
# df = pd.DataFrame(dict_data)

In [7]:
# df['lable'] = df['Token'].isin(mountains).apply(lambda x: int(x))

## Adding data from Kaggle NER dataset

In [8]:
ner_df = pd.read_csv('ner_dataset.csv', encoding= 'unicode_escape')

In [9]:
ner_df['Sentence #']=ner_df['Sentence #'].ffill()
ner_df=ner_df[['Sentence #','Word','Tag']]
ner_df.rename(columns={'Sentence #':'Sentence','Word':'Token','Tag':'lable'},inplace=True)

In [10]:
ner_df

,Sentence,Token,lable
0,Sentence: 1,Thousands,O
1,Sentence: 1,of,O
2,Sentence: 1,demonstrators,O
3,Sentence: 1,have,O
4,Sentence: 1,marched,O
...,...,...,...
1048570,Sentence: 47959,they,O
1048571,Sentence: 47959,responded,O
1048572,Sentence: 47959,to,O
1048573,Sentence: 47959,the,O


In [11]:
words = []
with open('list_of_mountains.txt','r') as file_obj: 
    for line in file_obj:
        if line.strip()!="":
            mountains = line.strip().split()
            for m in mountains:
                words.append(m)

In [12]:
ner_df[ner_df['Token'].isin(words)]

,Sentence,Token,lable
15777,Sentence: 713,Moses,B-per
149887,Sentence: 6827,Everest,I-geo
149948,Sentence: 6831,Everest,I-geo
149971,Sentence: 6832,Everest,I-geo
406541,Sentence: 18573,Everest,I-org
515430,Sentence: 23572,Everest,I-geo
515479,Sentence: 23574,Everest,I-geo
515481,Sentence: 23575,Everest,B-geo
515526,Sentence: 23577,Everest,B-org
528558,Sentence: 24177,Alps,I-geo


In [13]:
raws = ner_df[ner_df['Token'].isin(words)][(ner_df['lable']=='I-geo') | (ner_df['lable']=='B-geo')]
raws.lable = '1'
ner_df.update(raws)

/tmp/ipykernel_15423/766366989.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  raws = ner_df[ner_df['Token'].isin(words)][(ner_df['lable']=='I-geo') | (ner_df['lable']=='B-geo')]


In [14]:
add_raws = ner_df[ner_df['Sentence'].isin(ner_df[ner_df['Token'].isin(words)]['Sentence'])]

In [15]:
add_raws.values

array([['Sentence: 713', 'Moses', 'B-per'],
       ['Sentence: 713', 'Bittok', 'I-per'],
       ['Sentence: 713', 'probably', 'O'],
       ...,
       ['Sentence: 42765', 'National', 'I-org'],
       ['Sentence: 42765', 'Monument', 'I-org'],
       ['Sentence: 42765', '.', 'O']], dtype=object)

In [16]:
np.shape(add_raws.values)

(439, 3)

In [17]:
max_sentense = np.max(sentenses_num)
max_sentense

186

In [18]:
sentence_name = ''
for raw in add_raws.values:
    if raw[0]!=sentence_name:
        sentence_name = raw[0]
        max_sentense+=1
        
    sentenses_num.append(max_sentense)
    words_list.append(raw[1])
    labels.append(1 if raw[2]=='1' else 0)

## Creating dataset

In [19]:
dict_data = {}
dict_data['Sentence']=sentenses_num
dict_data['Word']=words_list
dict_data['Label']=labels

In [20]:
df = pd.DataFrame(dict_data)

In [21]:
df

,Sentence,Word,Label
0,1,The,0
1,1,Rocky,1
2,1,Mountains,0
3,1,is,0
4,1,an,0
...,...,...,...
4441,205,Papahanaumokuakea,0
4442,205,Marine,0
4443,205,National,0
4444,205,Monument,0


In [22]:
df.to_csv('dataset.csv', sep=',', header=True, index=None, encoding='utf-8') 